In [13]:
import torch
import config
from utils import (
    get_model_instance_segmentation,
    collate_fn,
    get_transform,
    myOwnDataset,
)
import model
import detect_utils
import cv2
import json
import matplotlib.pyplot as plt



In [10]:

print("Torch version:", torch.__version__)

# create own Dataset
training_dataset = myOwnDataset(
    root=config.data_dir, annotation=config.train_coco, transforms=get_transform()
)

# training DataLoader
training_loader = torch.utils.data.DataLoader(
    training_dataset,
    batch_size=config.train_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)


# select device (whether GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# DataLoader is iterable over Dataset
for imgs, annotations in training_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)


model = model.get_model(device = device, model_name='v2')


# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params, lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay
)

len_dataloader = len(training_loader)


Torch version: 1.13.1+cpu
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
[{'boxes': tensor([[329.9900, 223.8900, 437.5500, 275.2000],
        [386.2500, 512.8500, 481.3500, 567.9000]]), 'labels': tensor([31, 21]), 'image_id': tensor([799]), 'area': tensor([5518.9038, 5235.2549]), 'iscrowd': tensor([0, 0])}]
[{'boxes': tensor([]), 'labels': tensor([], dtype=torch.int64), 'image_id': tensor([1195]), 'area': tensor([]), 'iscrowd': tensor([], dtype=torch.int64)}]
[{'boxes': tensor([[ 509.3300,  550.0300,  554.9600,  594.0600],
        [1112.9500,  226.2600, 1169.7900,  273.0200]]), 'labels': tensor([ 7, 18]), 'image_id': tensor([604]), 'area': tensor([2009.0889, 2657.8384]), 'iscrowd': tensor([0, 0])}]
[{'boxes': tensor([[803.5000, 282.8600, 938.0100, 353.8900],
        [554.6500, 342.4600, 691.5700, 462.1600]]), 'labels': tensor([ 9, 10]), 'image_id': tensor([1060]), 'area': tensor([ 9554.2451, 16389.3242]), 'iscrowd': tensor([0, 0])}]
[{'boxes': tensor

In [11]:

# Training
# for epoch in range(config.num_epochs):
for epoch in range(1):
    print(f"Epoch: {epoch}/{config.num_epochs}")
    model.train()
    i = 0
    
    for imgs, annotations in training_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        print("imgs:", imgs)
        print("num: ")
        print(annotations[0]['boxes'].size() == torch.Size([0]))
        
        #if the annotations are emptpy, skip
        if annotations[0]['boxes'].size() == torch.Size([0]):
            continue
        
        print("annotations:", annotations)
        # pass if there are no annotations
        # if len(annotations) == 0:
        #     continue
        # if 
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        print(f"Iteration: {i}/{len_dataloader}, Loss: {losses}")

Epoch: 0/10
imgs: [tensor([[[0.3373, 0.3373, 0.3373,  ..., 0.3725, 0.3725, 0.3725],
         [0.3373, 0.3373, 0.3373,  ..., 0.3725, 0.3725, 0.3725],
         [0.3373, 0.3373, 0.3373,  ..., 0.3725, 0.3725, 0.3725],
         ...,
         [0.3451, 0.3451, 0.3451,  ..., 0.3490, 0.3490, 0.3490],
         [0.3451, 0.3451, 0.3451,  ..., 0.3490, 0.3490, 0.3490],
         [0.3451, 0.3451, 0.3451,  ..., 0.3490, 0.3490, 0.3490]],

        [[0.4196, 0.4196, 0.4196,  ..., 0.4549, 0.4549, 0.4549],
         [0.4196, 0.4196, 0.4196,  ..., 0.4549, 0.4549, 0.4549],
         [0.4196, 0.4196, 0.4196,  ..., 0.4549, 0.4549, 0.4549],
         ...,
         [0.4275, 0.4275, 0.4275,  ..., 0.4471, 0.4471, 0.4471],
         [0.4275, 0.4275, 0.4275,  ..., 0.4471, 0.4471, 0.4471],
         [0.4275, 0.4275, 0.4275,  ..., 0.4471, 0.4471, 0.4471]],

        [[0.1176, 0.1176, 0.1176,  ..., 0.1608, 0.1608, 0.1608],
         [0.1176, 0.1176, 0.1176,  ..., 0.1608, 0.1608, 0.1608],
         [0.1176, 0.1176, 0.1176,  ...,

KeyboardInterrupt: 

In [15]:
#load json file

with open(config.test_coco) as f:
    json_data = json.load(f)



num_correct = 0
total = 0
# create test Dataset
testing_dataset = myOwnDataset(
    root=config.data_dir, annotation=config.test_coco, transforms=get_transform()
)
#testing DataLoader
testing_loader = torch.utils.data.DataLoader(
    testing_dataset,
    batch_size=config.test_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)


model.eval()

# Evaluate the model
with torch.no_grad():
    num_correct_boxes = 0
    num_correct_labels = 0
    total_boxes = 0

    for imgs, annotations in testing_loader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]

        for idx, image in enumerate(imgs):
            image_np = image.cpu().numpy().transpose(1, 2, 0)  # Convert image to NumPy array with the correct shape


            boxes, pred_classes, labels = detect_utils.predict(image_np, model, device, 0.8)
            print("Unique label values:", torch.unique(labels).cpu().numpy())

            gt_boxes = annotations[idx]['boxes'].cpu().numpy()
            gt_labels = annotations[idx]['labels'].cpu().numpy()

            for gt_box, gt_label in zip(gt_boxes, gt_labels):
                total_boxes += 1
                iou_threshold = 0.5

                for pred_box, pred_label in zip(boxes, pred_classes):
                    iou = detect_utils.calculate_iou(gt_box, pred_box)

                    if iou > iou_threshold:
                        num_correct_boxes += 1
                        if gt_label == pred_label:
                            num_correct_labels += 1
                        break

    box_accuracy = num_correct_boxes / total_boxes
    label_accuracy = num_correct_labels / total_boxes

    print(f"Bounding Box Accuracy: {box_accuracy * 100:.2f}%")
    print(f"Label Accuracy: {label_accuracy * 100:.2f}%")


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x00000233B5C9D5A0>
Traceback (most recent call last):
  File "C:\Users\thoma\AppData\Roaming\Python\Python310\site-packages\torch\utils\data\dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "C:\Users\thoma\AppData\Roaming\Python\Python310\site-packages\torch\utils\data\dataloader.py", line 1424, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


TypeError: 'module' object is not callable